# Teil 9 - Einleitung in verschlüsselte Programme

So komisch es auch klingen mag, es ist möglich auf verschlüsselten Daten zu rechnen. Mit anderen Worten: es ist möglich ein Programm zu entwickeln in dem **ALLE Variablen verschlüsselt** sind!

In diesem Tutorial werden die grundlegenden Werkzeuge fürs verschlüsselte Rechnen behandelt. Im Speziellen wird der Ansatz des "Secure Multi-Party Computation" (Sicheres Rechnen mit mehreren Beteiligten) betrachtet.  
In diesem Abschnitt wird eine Rechenmaschine entwickelt, welche mit verschlüsselten Zahlen umgehen kann. 

Autoren:
- Andrew Trask - Twitter: [@iamtrask](https://twitter.com/iamtrask)
- Théo Ryffel - GitHub: [@LaRiffle](https://github.com/LaRiffle)

Referenzen: 
- Morten Dahl - [Blog](https://mortendahl.github.io) - Twitter: [@mortendahlcs](https://twitter.com/mortendahlcs)

Übersetzer:
- Jan Moritz Behnken - Github: [@JMBehnken](https://github.com/JMBehnken)

# Schritt 1: Verschlüsseln mit Secure Multi-Party Computation

SMPC ist auf den ersten Blick eine sonderbare Form der "Verschlüsselung". Anstelle eines öffentlichen/privaten Schlüssels für die Verschlüsselung der Variable, wird der Wert in mehrere `shares` aufgeteilt. Diese fungieren dann als private Schlüssel. Typischerweise werden diese `shares` an zwei oder mehrere _Besitzer_ verteilt. Somit müssen alle Besitzer der Variable einer Entschlüsselung zustimmen. Im Kern besitzt damit jeder einen privaten Schlüssel. 

### Verschlüsseln()

Um nun die Variable `x` zu "verschlüsseln", kann folgendermaßen vorgegangen werden.

> Die Verschlüsselung nutzt keinerlei Floats oder Reale Zahlen, sondern findet im mathematischen Raum des [Integer Quotientenringes](http://mathworld.wolfram.com/QuotientRing.html) statt. Dieser umfasst alle Integer zwischen `0` und `Q-1`, wobei `Q` eine Primzahl ist, welche "groß genug" ist, um alle Zahlen in den Berechnungen einzuschließen. In der Praxis wird ein gegebener Integer Wert `x` mit `x % Q` dem Ring angepasst. Aus diesem Grund sollten Zahlen `x' > Q'` vermieden werden.

In [ ]:
Q = 1234567891011

In [ ]:
x = 25

In [ ]:
import random

def encrypt(x):
    share_a = random.randint(-Q,Q)
    share_b = random.randint(-Q,Q)
    share_c = (x - share_a - share_b) % Q
    return (share_a, share_b,  share_c)

In [ ]:
encrypt(x)

Wie zu sehen ist, wurde die Variable `x` in drei unterschiedliche Anteile aufgespalten. Diese Anteile könnten an drei unterschiedliche Besitzer versendet werden.

### Entschlüsseln()

Sollen die drei Anteil entschlüsselt werden, so reicht es aus sie aufzusummieren und den Modulus zu berechnen (mod Q).

In [ ]:
def decrypt(*shares):
    return sum(shares) % Q

In [ ]:
a,b,c = encrypt(25)

In [ ]:
decrypt(a, b, c)

Anzumerken ist, dass ein Entschlüsseln mit nur zwei Anteilen nicht funktioniert!

In [ ]:
decrypt(a, b)

Somit werden alle Besitzer benötigt um den Wert entschlüsseln zu können. Genau deshalb verhalten sich die Anteile wie private Schlüssel, welche ebenfalls alle anwesend sein müssen, um eine Entschlüsselung zu ermöglichen.

# Schritt 2: Grundlegende Arithmetik mit SMPC

Die herausragende Eigenschaft von Secure Multi-Party Computation ist, dass **Berechnungen möglich sind während die Variablen verschlüsselt bleiben**. Simple Addition wird hier als Beispiel heran gezogen.

In [ ]:
x = encrypt(25)
y = encrypt(5)

In [ ]:
def add(x, y):
    z = list()
    # the first worker adds their shares together
    z.append((x[0] + y[0]) % Q)
    
    # the second worker adds their shares together
    z.append((x[1] + y[1]) % Q)
    
    # the third worker adds their shares together
    z.append((x[2] + y[2]) % Q)
    
    return z

In [ ]:
decrypt(*add(x,y))

### Erfolg!!!

Es ist vollbracht! Wenn jeder Helfer für sich seine Anteile addiert, lassen sich die abgeänderten Anteile zum korrekten Ergebnis entschlüsseln (25 + 5 = 30).

Wie sich herausstellte, können die SMPC Protokolle verwendet werden um folgende verschlüsselte Berechnungen zu ermöglichen:
- Addition
- Multiplikation
- Vergleiche

Und mit diesen grundlegenden Rechenarten, lässt sich jede erdenkliche Berechnung aufstellen!!!

Im nächsten Abschnitt werden diese Operationen in der PySyft Bibliothek erläutert!

# Schritt 3: SMPC mit PySyft

In den vorangegangenen Abschnitten wurden die grundlegenden Berechnungen für SMPC vorgestellt. In der Praxis soll es jedoch nicht darum gehen diese Funktionen für eigene verschlüsselte Programme regelmäßig selbst implementieren zu müssen. Deshalb handelt dieser Abschnitt von der Umsetzung der verschlüsselten Berechnung in PySyft. Insbesondere wird es um die drei Grundlagen gehen:

- addieren
- multiplizieren
- vergleichen

Zuerst werden dafür einige virtuelle Helfer (bekannt aus früheren Tutorials) erstellt.

In [ ]:
import torch
import syft as sy
hook = sy.TorchHook(torch)

bob = sy.VirtualWorker(hook, id="bob")
alice = sy.VirtualWorker(hook, id="alice")
bill = sy.VirtualWorker(hook, id="bill")

### Grundlegendes Verschlüsseln/Entschlüsseln

Fürs Verschlüsseln reicht es aus auf irgendeinem PySyft Tensor `.share()` aufzurufen.  
Fürs Entschlüsseln genügt ein Aufruf von `.get()` auf der geteilten Variable.

In [ ]:
x = torch.tensor([25])

In [ ]:
x

In [ ]:
encrypted_x = x.share(bob, alice, bill)

In [ ]:
encrypted_x.get()

### Prüfen der verschlüsselten Werte

Werden die Helfer Bob, Alice und Bill genauer betrachtet, so können die erstellten Anteile eingesehen werden.

In [ ]:
bob._objects

In [ ]:
x = torch.tensor([25]).share(bob, alice, bill)

In [ ]:
# Bob's share
bobs_share = list(bob._objects.values())[0]
bobs_share

In [ ]:
# Alice's share
alices_share = list(alice._objects.values())[0]
alices_share

In [ ]:
# Bill's share
bills_share = list(bill._objects.values())[0]
bills_share

Wenn nötig, lassen sich die Werte auf DIESELBE Art entschlüsseln wie oben!!!

In [ ]:
Q = x.child.field

(bobs_share + alices_share + bills_share) % Q

Beim Aufrufen von `.share()` wurde der Tensor in drei Anteile aufgeteilt und anschließend auf alle Beteiligten verteilt!

# Verschlüsselte Arithmetik

Nun wird die Arithmetik auf den zugrundeliegenden Daten vorgestellt! Die API ist dabei so aufgebaut, dass sie genau wie bei normalen PyTorch Tensoren verwendet werden kann.

In [ ]:
x = torch.tensor([25]).share(bob,alice)
y = torch.tensor([5]).share(bob,alice)

In [ ]:
z = x + y
z.get()

In [ ]:
z = x - y
z.get()

# Verschlüsselte Multiplikation

Für die Multiplikation wird ein weiterer Beteiligter benötigt, der verlässlich zufällige Zahlen bereitstellt (und keinen Beteiligten bevorzugt). Dieser Beteiligte wird "Crypto Provider" genannt. Für alle folgenden Fälle ist der "Crypto Provider" nur ein weiterer virtueller Helfer, hervorzuheben ist dabei jedoch, dass er zu keinem Zeitpunkt ein Besitzer irgendwelcher Anteile ist. Es ist darauf zu achten, dass der "Crypto Provider" verlässlich arbeiten muss, da die Berechnungen sonst angreifbar werden.

In [ ]:
crypto_provider = sy.VirtualWorker(hook, id="crypto_provider")

In [ ]:
x = torch.tensor([25]).share(bob,alice, crypto_provider=crypto_provider)
y = torch.tensor([5]).share(bob,alice, crypto_provider=crypto_provider)

In [ ]:
# multiplication

z = x * y
z.get()

Auch Matrixmultiplikation ist möglich.

In [ ]:
x = torch.tensor([[1, 2],[3,4]]).share(bob,alice, crypto_provider=crypto_provider)
y = torch.tensor([[2, 0],[0,2]]).share(bob,alice, crypto_provider=crypto_provider)

In [ ]:
# matrix multiplication

z = x.mm(y)
z.get()

# Verschlüsselte Vergleiche

Es ist ebenfalls möglich vertrauliche Vergleiche mit verschlüsselten Daten durchzuführen. Dafür wird das SecureNN Protokoll verwendet, welches [hier](https://eprint.iacr.org/2018/442.pdf) genauer studiert werden kann. Das Ergebnis des Vergleiches ist erneut ein vertraulicher Tensor.

In [ ]:
x = torch.tensor([25]).share(bob,alice, crypto_provider=crypto_provider)
y = torch.tensor([5]).share(bob,alice, crypto_provider=crypto_provider)

In [ ]:
z = x > y
z.get()

In [ ]:
z = x <= y
z.get()

In [ ]:
z = x == y
z.get()

In [ ]:
z = x == y + 20
z.get()

Auch die `max` Operation ist möglich.

In [ ]:
x = torch.tensor([2, 3, 4, 1]).share(bob,alice, crypto_provider=crypto_provider)
x.max().get()

In [ ]:
x = torch.tensor([[2, 3], [4, 1]]).share(bob,alice, crypto_provider=crypto_provider)
max_values, max_ids = x.max(dim=0)
max_values.get()

# Herzlichen Glückwunsch!!! - Zeit, der Community beizutreten! 

Herzlichen Glückwunsch zum Abschluss dieses Notebook-Tutorials! Wenn es Ihnen gefallen hat und Sie sich der Bewegung zur Wahrung der Privatsphäre, zum dezentralisiertenen Besitz von KI und der KI-Lieferkette (Daten) anschließen möchten, können Sie dies auf folgende Weise tun! 

### PySyft auf GitHub einen Stern geben! 

Der einfachste Weg, unserer Community zu helfen, besteht darin, die GitHub-Repos mit Sternen auszuzeichnen! Dies hilft, das Bewusstsein für die coolen Tools zu schärfen, die wir bauen. 

- [Gib PySyft einen Stern](https://github.com/OpenMined/PySyft)

### Mach mit bei Slack! 

Der beste Weg, um über die neuesten Entwicklungen auf dem Laufenden zu bleiben, ist, sich unserer Community anzuschließen! Sie können dies tun, indem Sie das Formular unter [http://slack.openmined.org](http://slack.openmined.org) ausfüllen.

### Treten Sie einem Code-Projekt bei! 

Der beste Weg, um zu unserer Community beizutragen, besteht darin, Entwickler zu werden! Sie können jederzeit zur PySyft GitHub Issues-Seite gehen und nach "Projects" filtern. Dies zeigt Ihnen alle Top-Level-Tickets und gibt einen Überblick darüber, an welchen Projekten Sie teilnehmen können! Wenn Sie nicht an einem Projekt teilnehmen möchten, aber ein wenig programmieren möchten, können Sie auch nach weiteren "einmaligen" Miniprojekten suchen, indem Sie nach GitHub-Problemen suchen, die als "good first issue" gekennzeichnet sind. 

- [PySyft Projects](https://github.com/OpenMined/PySyft/issues?q=is%3Aopen+is%3Aissue+label%3AProject)
- [Good First Issue Tickets](https://github.com/OpenMined/PySyft/issues?q=is%3Aopen+is%3Aissue+label%3A%22good+first+issue%22)

### Spenden

Wenn Sie keine Zeit haben, zu unserer Codebase beizutragen, aber dennoch Unterstützung leisten möchten, können Sie auch Unterstützer unseres Open Collective werden. Alle Spenden fließen in unser Webhosting und andere Community-Ausgaben wie Hackathons und Meetups! 

 - [OpenMined's Open Collective Page](https://opencollective.com/openmined)